In [23]:
from __future__ import division
from mnist import MNIST
from sklearn import tree
import numpy as np
from sklearn.metrics import classification_report,log_loss,accuracy_score,roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from time import time
import pandas as pd
from datetime import datetime
from openpyxl import load_workbook
import re
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from mlxtend.data import loadlocal_mnist
import timeit
import timeit 
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import confusion_matrix


images_train_o,labels_train_o  = loadlocal_mnist(
        images_path='/users/mtech/jatindev/Downloads/train-images.idx3-ubyte',
        labels_path='/users/mtech/jatindev/Downloads/train-labels.idx1-ubyte')
images_test_o,labels_test_o= loadlocal_mnist(
        images_path='/users/mtech/jatindev/Downloads/t10k-images.idx3-ubyte',
        labels_path='/users/mtech/jatindev/Downloads/t10k-labels.idx1-ubyte')



In [24]:
excel_file_name = 'LinearSvm' + str(datetime.now().strftime('%Y%m%d%H%M%S')) + '.xlsx'
excel_writer = pd.ExcelWriter(excel_file_name, engine='openpyxl')
df = pd.DataFrame(columns=['Examples','Accuracy','Normalized Accuracy', 
                     'Time_taken_train','Time_taken_test',
                     'Negative Log loss','Macro_auc','Micro_auc'])
df.to_excel(excel_writer, sheet_name='demo_sheet')

excel_writer.save()


/usr/lib/python3/dist-packages/pandas/io/excel.py:687: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])


In [25]:
def classifaction_report_to_dataframe(report):
    report_data = []
    lines = report.split('\n')
    for line in lines[2:-3]:
        row = {}
        row_data = line.split('      ')
        print(line)
        if len(row_data)>5:
            row['class'] = row_data[1]
            row['precision'] = float(row_data[2])
            row['recall'] = float(row_data[3])
            row['f1_score'] = float(row_data[4])
            row['support'] = float(row_data[5])
            report_data.append(row)
    for line in lines[-2:-1]:
        row = {}
        p = re.compile("weighted avg")
        m = p.search(line)  
        print(line)
        row['class'] = m.group(0)
        p = re.compile("\d+.?\d+")
        row_data = p.findall(line)
        row['precision'] = float(row_data[0])
        row['recall'] = float(row_data[1])
        row['f1_score'] = float(row_data[2])
        row['support'] = float(row_data[3])
        report_data.append(row)
        
    dataframe = pd.DataFrame.from_dict(report_data)
    return dataframe

In [26]:
for t in[100,200,500,1000,2000,5000,10000,20000,30000,40000,50000,60000]:
    print('For t=',t)
    images_train=images_train_o[:t]
    labels_train=labels_train_o[:t]
    images_test=images_test_o[0:t]
    labels_test=labels_test_o[0:t]
    svm = LinearSVC(dual=True)
    classifier= CalibratedClassifierCV(svm)
    start=time()
    classifier.fit(images_train,labels_train)
    train_time=time()-start
    start=time()
    predictions=classifier.predict(images_test)
    test_time=time()-start
    accuracy=accuracy_score(predictions,labels_test,normalize=False)
    normalized_accuracy=accuracy_score(predictions,labels_test,normalize=True)
    log_loss_prediction=classifier.predict_proba(images_test)
    log_loss_val=log_loss(labels_test,log_loss_prediction)
    classification_repo=classification_report(labels_test,predictions)

    labels_test_one_hot=label_binarize(labels_test,classes=[0,1,2,3,4,5,6,7,8,9])
    predictions_one_hot=label_binarize(predictions,classes=[0,1,2,3,4,5,6,7,8,9])
    micro_auc=roc_auc_score(labels_test_one_hot,predictions_one_hot,average="micro")
    macro_auc=roc_auc_score(labels_test_one_hot,predictions_one_hot,average="macro")

    #print(micro_auc,macro_auc)
    cm = confusion_matrix(labels_test,predictions)
    cm_df = pd.DataFrame(data=cm)
    df_data = [t,accuracy,normalized_accuracy,train_time,test_time,log_loss_val,macro_auc,micro_auc]
    df.loc[len(df)] = df_data   

    book = load_workbook(excel_file_name)
    with pd.ExcelWriter(excel_file_name, engine='openpyxl') as excel_writer:
        excel_writer.book = book
        df_classification_report = classifaction_report_to_dataframe(classification_repo)
        df_classification_report.to_excel(excel_writer, sheet_name='Linearsvm_cr'+str(t))

        cm_df.to_excel(excel_writer, sheet_name='Linearsvm_cr'+str(t),
                       startrow=len(df_classification_report)+5)

        excel_writer.save()

book = load_workbook(excel_file_name)
with pd.ExcelWriter(excel_file_name, engine='openpyxl') as excel_writer:
    excel_writer.book = book
    df.to_excel(excel_writer, sheet_name='Linearsvm')
    excel_writer.save()



For t= 100


/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


           0       0.89      1.00      0.94         8
           1       0.70      1.00      0.82        14
           2       0.80      0.50      0.62         8
           3       0.77      0.91      0.83        11
           4       0.50      0.86      0.63        14
           5       1.00      0.14      0.25         7
           6       1.00      0.50      0.67        10
           7       0.92      0.80      0.86        15
           8       0.50      0.50      0.50         2
           9       0.62      0.45      0.53        11

   micro avg       0.72      0.72      0.72       100
weighted avg       0.77      0.72      0.70       100
For t= 200


/usr/lib/python3/dist-packages/pandas/io/excel.py:687: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])
/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/lib/python3/dist-packages/pandas/io/excel.py:687: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])
/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, Fut

           0       0.89      0.94      0.91        17
           1       0.97      1.00      0.98        28
           2       0.65      0.69      0.67        16
           3       0.54      0.88      0.67        16
           4       0.61      0.71      0.66        28
           5       1.00      0.05      0.10        20
           6       0.71      0.60      0.65        20
           7       0.73      0.79      0.76        24
           8       0.73      0.80      0.76        10
           9       0.59      0.62      0.60        21

   micro avg       0.71      0.71      0.71       200
weighted avg       0.75      0.71      0.68       200
For t= 500


...

/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/lib/python3/dist-packages/pandas/io/excel.py:687: 

           0       0.95      0.97      0.96       980
           1       0.94      0.99      0.96      1135
           2       0.93      0.87      0.90      1032
           3       0.89      0.89      0.89      1010
           4       0.91      0.93      0.92       982
           5       0.87      0.85      0.86       892
           6       0.92      0.95      0.94       958
           7       0.91      0.90      0.91      1028
           8       0.86      0.82      0.84       974
           9       0.88      0.88      0.88      1009

   micro avg       0.91      0.91      0.91     10000
weighted avg       0.91      0.91      0.91     10000
For t= 60000


/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/lib/python3/dist-packages/pandas/io/excel.py:687: 

           0       0.95      0.98      0.96       980
           1       0.95      0.99      0.97      1135
           2       0.92      0.87      0.90      1032
           3       0.89      0.89      0.89      1010
           4       0.91      0.92      0.92       982
           5       0.85      0.84      0.84       892
           6       0.93      0.94      0.93       958
           7       0.92      0.92      0.92      1028
           8       0.85      0.83      0.84       974
           9       0.88      0.88      0.88      1009

   micro avg       0.91      0.91      0.91     10000
weighted avg       0.91      0.91      0.91     10000


/usr/lib/python3/dist-packages/pandas/io/excel.py:687: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])
